# Visualização

As tabelas de dados (`DataFrame`) da biblioteca [pandas](https://pandas.pydata.org/) permitem que conjuntos de dados sejam estruturados e visualizados numa forma tabelada. No entanto, em muitas situações, visualizações gráficas são mais apelativas, claras e interpretáveis.

Em Python, a biblioteca [matplotlib](https://matplotlib.org/) é a mais usada para visualização gráfica:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(0, 10, 100)

fig = plt.figure()
plt.plot(x, np.sin(x), '-')
plt.plot(x, np.cos(x), '--');

**Nota**: O `;` no final da instrução é algo estranho em Python, mas não é um erro. Neste caso, serve apenas para que o retorno da função não seja mostrado no notebook.

Apesar de a biblioteca *matplotlib* poder ser usada para gerar a maioria das visualizações de dados de interesse, esta nāo está preparada para lidar diretamente com as estruturas de dados da biblioteca *pandas*, o que torna o processo de geração das visualizações bastante moroso. Para lidar com esta limitação, a biblioteca *pandas* implementa alguns métodos de visualização básicos. A biblioteca [Seaborn](http://seaborn.pydata.org/), para além dos básicos, implementa alguns métodos para gerar visualizações mais complexas, mas que são bastante úteis no contexto da análise de dados.

**Nota**: Quer os métodos de visualização da biblioteca *pandas*, quer a biblioteca *Seaborn* são construídos em cima da biblioteca *matplotlib*.

**Nota**: Neste tutorial não se pretende explorar ao detalhe como gerar cada tipo de visualização, mas sim mostrar algumas visões possíveis sobre os dados.

## pandas plot

Para explorar os métodos de visualização da biblioteca *pandas* vamos carregar o conjunto de dados de filmes que processamos anteriormente:

In [ ]:
import pandas as pd
import os

data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/TheAwesomeGe/DECD/main/data/'

movies_df = pd.read_csv(data_path + 'IMDB-Movie-Data-Processed.csv', index_col='Title')
movies_df.head()

O principal método para visualização fornecido pela biblioteca *pandas* é o método `plot`. Este recebe um argumento `kind` que define o tipo de gráfico. Por exemplo:

- `'line'`: gráfico de linha
- `'bar'` e `'barh'`: gráficos de barras (verticais ou horizontais)
- `'pie'`: gráfico de pizza
- `'hist'`: histograma
- `'density'`: densidade de probabilidade
- `'scatter'`: gráfico de dispersão
- `'box'`: diagrama de caixa

Para começar, vamos observar a relação entre a classificação dos filmes e a sua receita usando um gráfico de dispersão:

In [ ]:
movies_df.plot(kind='scatter', x='rating', y='revenue_millions', title='Revenue (millions) vs Rating');

O método `plot` também pode ser aplicado sobre uma série. Por exemplo, para obter um histograma das classificações dos filmes:

In [ ]:
movies_df['rating'].plot(kind='hist', title='Rating');

Anteriormente, usámos o método `describe` para obter um resumo da distribuição dos valores dum atributo:

In [ ]:
movies_df['rating'].describe()

Uma representação gráfica desta informação pode ser obtida usando um diagrama de caixa:

![diagrama de caixa](../images/boxplot.svg)

In [ ]:
movies_df['rating'].plot(kind='box');

Neste contexto, a biblioteca *pandas* disponibiliza outro método, `boxplot`, que pode ser aplicado sobre uma tabela de dados para obter uma visualização na forma de diagramas de caixa da distribuição dum atributo contínuo para cada valor de um atributo discreto. Por exemplo, podemos usar este método para visualizar a diferença das receitas de filmes "bons" e "maus":  

In [ ]:
movies_df.boxplot(column='revenue_millions', by='rating_category');

O método `plot` também pode ser usado para visualizar a distribuição de valores de atributos categóricos. Por exemplo, podemos usar um gráfico de pizza para analisar a proporção de filmes "bons" e "maus":

In [ ]:
movies_df['rating_category'].value_counts().plot(kind='pie');

Ou um gráfico de barras para visualizar os dez realizadores com mais filmes no conjunto de dados:

In [ ]:
movies_df['director'].value_counts().head(10).plot(kind='bar');

Informação detalhada sobre a funcionalidade do método `plot` pode ser obtida na [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) da biblioteca *pandas*.

## Seaborn

A biblioteca [Seaborn](http://seaborn.pydata.org/) disponibiliza uma interface de alto nível para a criação de gráficos úteis para fazer uma análise estatística de dados e que integra bem com as estruturas de dados da biblioteca *pandas*. 

Normalmente, a biblioteca *Seaborn* é importada como `sns`:

In [ ]:
import seaborn as sns
sns.set_theme()  # set the chart style

Tal como no caso do método `plot` da biblioteca *pandas*, não vamos explorar em detalhe toda a funcionalidade disponibilizada pela biblioteca *Seaborn*, mas sim explorar apenas alguns exemplos de gráficos que podem ser úteis para a análise de dados. Neste contexto, vamos também olhar para alguns conjuntos de dados que são muito usados na introdução à ciência de dados e que são disponibilizados pela própria biblioteca *Seaborn*.

### Histogramas e densidade de probabilidade

Durante a fase inicial da exploração de um novo conjunto de dados, muitas vezes queremos apenas ter uma noção da distribuição dos valores das variáveis desse conjunto.  

Como exemplo, vamos criar um conjunto de dados fazendo uma amostragem aleatória de uma distribuição normal:

In [ ]:
data = np.random.multivariate_normal([0, 0], [[5, 2], [2, 2]], size=2000)
data = pd.DataFrame(data, columns=['x', 'y'])
data.head()

A função `histplot` pode ser usada para gerar um histograma destes dados:

In [ ]:
sns.histplot(data);

Também podemos visualizar a distribuição de cada uma das variáveis através duma aproximação da função de densidade de probabilidade usando a função `kdeplot`:

In [ ]:
sns.kdeplot(data=data, fill=True);

Usando a mesma função, podemos visualizar a distribuição da probablidade conjunta de duas variáveis, passando os seus nomes aos argumentos `x` e `y`:

In [ ]:
sns.kdeplot(data=data, x='x', y='y', fill=True);

A função `jointplot` permite-nos visualizar ao mesmo tempo a distribuição da probablidade conjunta de duas variáveis e a distruição marginal de cada uma delas:

In [ ]:
sns.jointplot(data=data, x='x', y='y', kind='hex');

Com esta função podemos também visualizar uma regressão linear sobre essas variáveis:

In [ ]:
sns.jointplot(data=data, x='x', y='y', kind='reg');

### Gráficos de pares

Como vimos para o conjunto de dados sobre filmes, quando os conjuntos de dados têm uma dimensionalidade maior, é interessante olhar para pares de atributos de forma a encontrar potenciais correlações entre eles.

Para este exemplo, vamos olhar para o conjunto de dados [Iris](https://archive.ics.uci.edu/dataset/53/iris), que contém amostras de três tipos de lírio (flor) caracterizadas pelas dimensões das suas sépalas e pétalas:

In [ ]:
iris = sns.load_dataset('iris')
iris.head()

A função `pairplot` pode ser usada para gerar um gráfico de pares entre as quatro características:

In [ ]:
sns.pairplot(iris, hue='species');

### Mapas de Calor

Uma forma comum de visualizar a correlação entre os atributos de um conjunto de dados é usar um mapa de calor. Podemos usar a função `heatmap` para obter uma visualização deste género para a matriz de correlação obtida usando o método `corr` da biblioteca *pandas*:

In [ ]:
sns.heatmap(iris.corr(numeric_only=True));

### Histogramas segmentados

Por vezes, conseguimos obter informação importante sobre um conjunto de dados olhando para os histogramas de um determinado atributo segmentado de acordo com os valores de outros atributos.

Por exemplo, vamos olhar para um conjunto de dados com informações sobre as gorjetas de um trabalhador de um restaurante:

In [ ]:
tips = sns.load_dataset('tips')
tips.head()

Vamos criar um novo atributo com os valores da percentagem da conta a que correspondem as gorjetas:

In [ ]:
tips['tip_percentage'] = 100 * tips['tip'] / tips['total_bill']
tips.head()

Podemos usar uma grelha (`FacetGrid`) para visualizar os histogramas deste atributo segmentado de acordo com a refeição e o género de quem paga a conta:

In [ ]:
g = sns.FacetGrid(tips, row='sex', col='time', margin_titles=True)
g.map(sns.histplot, 'tip_percentage');

Esta visão em grelha permite-nos obter rapidamente alguma informação interessante. Por exemplo:

- A maioria dos dados deste conjunto correspondem a contas pagas por homens ao jantar
- Na maioria dos casos, as gorjetas são entre 10% e 20% do valor da conta

### Gráficos categóricos

Os gráficos categóricos (`catplot`) definidos pela biblioteca *Seaborn* também são úteis para este tipo de visualização, já que permitem, por exemplo, observar a distribuição de um determinado atributo segmentado de acordo com os valores de outro atributo:

In [ ]:
with sns.axes_style(style='ticks'):
    g = sns.catplot(data=tips, x='day', y='total_bill', hue='sex', kind='box')
    g.set_axis_labels('Day', 'Total Bill');

O argumento `kind` da função `catplot` pode ser usado para criar outros tipos de visualização. Para isso, vamos olhar para um conjunto de dados sobre a descoberta de novos planetas:

In [ ]:
planets = sns.load_dataset('planets')
planets.head()

Podemos usar a função `catplot` com o argumento `kind='count'` para analisar a quantidade de planetas descobertos ao longo dos anos:  

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot(data=planets, x='year', kind='count', aspect=2, color='steelblue')
    g.set_xticklabels(step=5)

Esta visualização pode tornar-se mais informativa se segmentar as contagens de acordo com o método usado para descobrir os planetas:

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot(data=planets, x='year', kind='count', hue='method', aspect=3.0, order=range(2001, 2015))
    g.set_ylabels('Number of Planets Discovered')

Informação detalhada sobre a funcionalidade da biblioteca *Seaborn* pode ser obtida na sua [documentação](https://seaborn.pydata.org/api.html). A sua [galeria de exemplos](https://seaborn.pydata.org/examples/index.html) pode ser usada como guia para encontrar tipos de visualização interessantes para um determinado problema.